In [502]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn import over_sampling
from businessscoring import businessgain
from sklearn import preprocessing

In [503]:
from Feature_Engineering import feature_engineering

In [504]:
features = feature_engineering(j_flag=1)

In [505]:
le = preprocessing.LabelEncoder()
le.fit(features.Service_Type)
features.Service_Type = le.transform(features.Service_Type)
features.columns

Index(['Provider', 'Phys_Count', 'Patient_Count', 'Norm_Patient_Count',
       'Claim_Count', 'Norm_Claim_Count', 'Service_Type', 'Both_Service',
       'Inpatient', 'Outpatient', 'Inpatient_Count', 'Outpatient_Count',
       'Norm_Inpatient_Count', 'Norm_Outpatient_Count', 'DuplicateClaims',
       'Duplicate_Claims_Percent', 'AvgClaimDuration', 'Avg_Cost', 'Avg_Age',
       'Gender1', 'Gender2', 'Race1', 'Race2', 'Race3', 'Race5',
       'ChronicCond_Alzheimer_1', 'ChronicCond_Alzheimer_2',
       'ChronicCond_KidneyDisease_1', 'ChronicCond_KidneyDisease_2',
       'ChronicCond_Cancer_1', 'ChronicCond_Cancer_2',
       'ChronicCond_ObstrPulmonary_1', 'ChronicCond_ObstrPulmonary_2',
       'ChronicCond_Depression_1', 'ChronicCond_Depression_2',
       'ChronicCond_Diabetes_1', 'ChronicCond_Diabetes_2',
       'ChronicCond_IschemicHeart_1', 'ChronicCond_IschemicHeart_2',
       'ChronicCond_Osteoporasis_1', 'ChronicCond_Osteoporasis_2',
       'ChronicCond_rheumatoidarthritis_1',
     

In [506]:
features.columns
features['Norm_FraudNeighbors'] = features['FraudNeighbor_Count']/(features['NetworkDegree'])  
features = features.fillna(0)
#features['NoPhysicianCount'] = features['NoPhysicianCount']/features['Phys_Count'] 
features['Gender1'] = features['Gender1']/features['Patient_Count']  
features['Race1'] = features['Race1']/features['Patient_Count']
features['Race2'] = features['Race2']/features['Patient_Count']
features['Race3'] = features['Race3']/features['Patient_Count']
conditions=['ChronicCond_Alzheimer_2','ChronicCond_KidneyDisease_2','ChronicCond_Cancer_2',\
            'ChronicCond_ObstrPulmonary_2','ChronicCond_Depression_2','ChronicCond_Diabetes_2',\
            'ChronicCond_IschemicHeart_2','ChronicCond_Osteoporasis_2','ChronicCond_rheumatoidarthritis_2',\
            'ChronicCond_stroke_2']
for condition in conditions:
    features[condition] = features[condition]/features['Patient_Count']


In [507]:
features['PotentialFraud'] = features['PotentialFraud'].map({'No': 0, 'Yes': 1})
#features.set_index('Provider', inplace=True)

In [508]:
X = features.drop(['PotentialFraud', 'Inpatient','Outpatient','Provider','Both_Service'], axis=1)
y = features['PotentialFraud'] 

In [509]:
#create seperate dataset with dummy variables to add back after scaling
non_num = X[['isFraudConnected','Service_Type']]

#drop dummy variables
X = X.drop(['isFraudConnected','Service_Type'], axis =1)

In [510]:
#scale on the training set 
scaler = RobustScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=(X.columns), index=X.index)
X_scaled.head()

,Phys_Count,Patient_Count,Norm_Patient_Count,Claim_Count,Norm_Claim_Count,Inpatient_Count,Outpatient_Count,Norm_Inpatient_Count,Norm_Outpatient_Count,DuplicateClaims,...,ChronicCond_Osteoporasis_1,ChronicCond_Osteoporasis_2,ChronicCond_rheumatoidarthritis_1,ChronicCond_rheumatoidarthritis_2,ChronicCond_stroke_1,ChronicCond_stroke_2,NetworkDegree,NoPhysicianCount,FraudNeighbor_Count,Norm_FraudNeighbors
0,0.470588,-0.017544,-0.155665,-0.077922,-0.165527,1.0,-0.100000,1.538462,-1.538462,0.0,...,-0.111111,0.465059,0.055556,-0.292398,0.500000,-1.291474,0.333333,0.0,0.0,0.0
1,2.235294,1.614035,0.031527,1.311688,0.000000,12.4,0.614286,3.615385,-3.615385,0.0,...,1.111111,0.551181,1.388889,0.179937,1.166667,0.137313,0.000000,0.0,0.0,0.0
2,1.882353,1.982456,0.222660,1.532468,0.125855,0.0,1.742857,0.000000,0.000000,0.0,...,1.944444,-0.031453,1.888889,-0.006356,2.333333,-0.286929,0.666667,0.0,0.0,0.0
3,0.000000,8.245614,15.763547,14.727273,26.151847,0.0,16.257143,0.000000,0.000000,4.0,...,7.444444,0.167860,6.666667,0.313663,6.166667,0.117038,-0.333333,2.0,0.0,0.0
4,0.235294,0.578947,0.650246,0.532468,0.574555,0.6,0.600000,0.307692,-0.307692,0.0,...,0.500000,0.117601,0.500000,0.161323,1.166667,-0.713470,1.333333,0.0,1.0,0.2


In [511]:
#add back dummy variables to scaled dataset 
X_scaled = pd.concat((X_scaled, non_num), axis=1)
X_scaled.head(5)

,Phys_Count,Patient_Count,Norm_Patient_Count,Claim_Count,Norm_Claim_Count,Inpatient_Count,Outpatient_Count,Norm_Inpatient_Count,Norm_Outpatient_Count,DuplicateClaims,...,ChronicCond_rheumatoidarthritis_1,ChronicCond_rheumatoidarthritis_2,ChronicCond_stroke_1,ChronicCond_stroke_2,NetworkDegree,NoPhysicianCount,FraudNeighbor_Count,Norm_FraudNeighbors,isFraudConnected,Service_Type
0,0.470588,-0.017544,-0.155665,-0.077922,-0.165527,1.0,-0.100000,1.538462,-1.538462,0.0,...,0.055556,-0.292398,0.500000,-1.291474,0.333333,0.0,0.0,0.0,0.0,0
1,2.235294,1.614035,0.031527,1.311688,0.000000,12.4,0.614286,3.615385,-3.615385,0.0,...,1.388889,0.179937,1.166667,0.137313,0.000000,0.0,0.0,0.0,0.0,0
2,1.882353,1.982456,0.222660,1.532468,0.125855,0.0,1.742857,0.000000,0.000000,0.0,...,1.888889,-0.006356,2.333333,-0.286929,0.666667,0.0,0.0,0.0,0.0,2
3,0.000000,8.245614,15.763547,14.727273,26.151847,0.0,16.257143,0.000000,0.000000,4.0,...,6.666667,0.313663,6.166667,0.117038,-0.333333,2.0,0.0,0.0,0.0,2
4,0.235294,0.578947,0.650246,0.532468,0.574555,0.6,0.600000,0.307692,-0.307692,0.0,...,0.500000,0.161323,1.166667,-0.713470,1.333333,0.0,1.0,0.2,1.0,0


In [512]:
#use as cv=skf, ensures results are consistent in each partition
s_kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True);
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [513]:
# SMOTE upsampling for all models
oversample = over_sampling.SMOTE()
X_train_Bal, y_train_Bal = oversample.fit_resample(X_train, y_train)

y_train_Bal.value_counts()

1    3923
0    3923
Name: PotentialFraud, dtype: int64

In [514]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_ros, y_ros = ros.fit_sample(X_train, y_train)

### Examining VIF

In [515]:
from statsmodels.stats.outliers_influence import variance_inflation_factor  
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X_train_Bal.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [round(variance_inflation_factor(X_train_Bal.values, i),3)
                          for i in range(len(X_train_Bal.columns))] 
  
print(vif_data)

                              feature         VIF
0                          Phys_Count       3.806
1                       Patient_Count    2653.614
2                  Norm_Patient_Count      37.870
3                         Claim_Count  128823.422
4                    Norm_Claim_Count      44.560
5                     Inpatient_Count    1225.220
6                    Outpatient_Count  121974.013
7                Norm_Inpatient_Count  739627.259
8               Norm_Outpatient_Count  739632.715
9                     DuplicateClaims       3.991
10           Duplicate_Claims_Percent       1.410
11                   AvgClaimDuration       2.680
12                           Avg_Cost       7.118
13                            Avg_Age       1.067
14                            Gender1       1.132
15                            Gender2     757.883
16                              Race1       7.953
17                              Race2       6.223
18                              Race3       3.639


### Examining Correlation Matrix

In [516]:
# Correlation Matrix 
corr = features.corr().abs()
corr.style.background_gradient(cmap='coolwarm')

### Non-linear Models


### Random Forest

In [517]:
from sklearn import ensemble
randomForest = ensemble.RandomForestClassifier()

# fit RF model on SMOTE training set
randomForest.set_params(random_state=0)

randomForest.fit(X_train_Bal, y_train_Bal)

# baseline score
print('SMOTE: \nRF BaseLine Train Set Recall Score:', recall_score(y_train_Bal, randomForest.predict(X_train_Bal)))
print('RF BaseLine Validation Set Recall Score:', recall_score(y_test, randomForest.predict(X_test)))

SMOTE: 
RF BaseLine Train Set Recall Score: 1.0
RF BaseLine Validation Set Recall Score: 0.6633663366336634


In [518]:
# set the parameter grid
grid_para_forest = {
    'criterion': ['entropy'],
    'max_depth': range(1, 31),
    'n_estimators': [1,51,101,151]
}

# run the RF classifier through the grid search
grid_search_forest = ms.GridSearchCV(randomForest, grid_para_forest, scoring=businessscore, cv=s_kf, n_jobs=-1)
%time grid_search_forest.fit(X_train_Bal, y_train_Bal)

CPU times: user 8.06 s, sys: 378 ms, total: 8.43 s
Wall time: 6min 12s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': range(1, 31),
                         'n_estimators': [1, 51, 101, 151]},
             scoring=make_scorer(businessgain))

In [519]:
print("RF Grid Search Best params: ", grid_search_forest.best_params_)
best_RF = grid_search_forest.best_estimator_
print("RF Grid Search Training set Recall score: ", recall_score(y_train_Bal, best_RF.predict(X_train_Bal)))
print("RF Grid Search Validation set Recall score: ", f1_score(y_test, best_RF.predict(X_test)))
print("RF Grid Search Validation set Business score: ", businessgain(y_test,best_RF.predict(X_test)))
SMOTE_rf = businessgain(y_test,best_RF.predict(X_test))

RF Grid Search Best params:  {'criterion': 'entropy', 'max_depth': 14, 'n_estimators': 101}
RF Grid Search Training set Recall score:  1.0
RF Grid Search Validation set Recall score:  0.6016949152542372
RF Grid Search Validation set Business score:  0.31919806997883915


#### imblearn

In [271]:
randomForest.set_params(random_state=0)

randomForest.fit(X_ros, y_ros)

# baseline score
print('imblearn: \nRF BaseLine Train Set Recall Score:', recall_score(y_ros, randomForest.predict(X_ros)))
print('RF BaseLine Validation Set Recall Score:', recall_score(y_test, randomForest.predict(X_test)))

imblearn: 
RF BaseLine Train Set Recall Score: 1.0
RF BaseLine Validation Set Recall Score: 0.5742574257425742


In [272]:
# set the parameter grid
grid_para_forest = {
    'criterion': ['entropy'],
    'max_depth': range(1, 31),
    'n_estimators': range(10, 110, 10)
}

# run the RF classifier through the grid search
grid_search_forest = ms.GridSearchCV(randomForest, grid_para_forest, scoring=businessscore, cv=s_kf, n_jobs=-1)
%time grid_search_forest.fit(X_ros, y_ros)

CPU times: user 11.8 s, sys: 339 ms, total: 12.2 s
Wall time: 7min 45s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': range(1, 31),
                         'n_estimators': range(10, 110, 10)},
             scoring=make_scorer(businessgain))

In [273]:
print("RF Grid Search Best params: ", grid_search_forest.best_params_)
best_RF = grid_search_forest.best_estimator_
print("RF Grid Search Training set Recall score: ", recall_score(y_ros, best_RF.predict(X_ros)))
print("RF Grid Search Validation set Recall score: ", recall_score(y_test, best_RF.predict(X_test)))
print("RF Grid Search Validation set Business score: ", businessgain(y_test,best_RF.predict(X_test)))
imblearn_rf = businessgain(y_test,best_RF.predict(X_test))

RF Grid Search Best params:  {'criterion': 'entropy', 'max_depth': 24, 'n_estimators': 20}
RF Grid Search Training set Recall score:  1.0
RF Grid Search Validation set Recall score:  0.6039603960396039
RF Grid Search Validation set Business score:  0.142402502614099


#### comparing SMOTE and imblearn

In [429]:
print('SMOTE rf business score: ', SMOTE_rf, '\nimblearn rf business score: ', imblearn_rf)
list_of_tuples = list(zip(X_train_Bal.columns, best_RF.feature_importances_))
df = pd.DataFrame(list_of_tuples, columns = ['Feature', 'Score'])
df.sort_values(by='Score', ascending=False)

SMOTE rf business score:  0.40486554043649253 
imblearn rf business score:  0.142402502614099


,Feature,Score
23,Procedure_1_Count,0.123460
2,Claim_Count,0.078716
24,Procedure_2_Count,0.069692
25,Procedure_3_Count,0.068200
1,Patient_Count,0.063466
6,Avg_Cost,0.045019
36,Diagnosis_9_Count,0.043831
3,Norm_Outpatient_Count,0.039274
11,Race3,0.038412
35,Diagnosis_8_Count,0.028615


### XG BOOST 

In [520]:
from xgboost import XGBClassifier

#SMOTE
xgb = XGBClassifier(random_state=0)
xgb.fit(X_train_Bal, y_train_Bal)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [521]:
print('SMOTE: \nXGB BaseLine Train Set Recall Score:', recall_score(y_train_Bal, xgb.predict(X_train_Bal)))
print('XGB BaseLine Validation Set Recall Score:', recall_score(y_test, xgb.predict(X_test)))

SMOTE: 
XGB BaseLine Train Set Recall Score: 1.0
XGB BaseLine Validation Set Recall Score: 0.594059405940594


In [522]:
# parameter tuning
grid_para_xgb = {'scale_pos_weight': np.arange(start=600, stop=750, step=10),
                 'learning_rate': [1e-4,1e-3,5],
                 'n_estimators':  np.arange(start=10, stop=15, step=1),
                 'max_depth': np.arange(start=10, stop=15, step=1),
                 'subsample': [0.9]
                 } 

In [ ]:
grid_search_xgb = GridSearchCV(xgb, grid_para_xgb, scoring=businessscore, cv=s_kf, n_jobs=-1)
%time grid_search_xgb.fit(X_train_Bal, y_train_Bal)

In [ ]:
grid_search_xgb.best_params_

In [ ]:
best_xgb = grid_search_xgb.best_estimator_
print('XGB BaseLine Train Set Recall Score:', recall_score(y_train_Bal, best_xgb.predict(X_train_Bal)))
print('XGB BaseLine Validation Set Recall Score:', recall_score(y_test, best_xgb.predict(X_test)))
print('XGB BaseLine Validation Set business Score:', businessgain(y_test, best_xgb.predict(X_test)))
SMOTE_xgb = businessgain(y_test, best_xgb.predict(X_test))

#### imblearn

In [281]:
xgb = XGBClassifier(random_state=0)
xgb.fit(X_ros, y_ros)
print('imblearn: \nXGB BaseLine Train Set Recall Score:', recall_score(y_ros, xgb.predict(X_ros)))
print('XGB BaseLine Validation Set Recall Score:', recall_score(y_test, xgb.predict(X_test)))

imblearn: 
XGB BaseLine Train Set Recall Score: 1.0
XGB BaseLine Validation Set Recall Score: 0.6336633663366337


In [282]:
# parameter tuning
grid_para_xgb = {'scale_pos_weight': [400,500,600],
                 'learning_rate': [0.001, 0.010, 0.1],
                 'n_estimators': [2,5,10] ,
                 'max_depth': [1,3,10],
                 'subsample': [0.9]
                 }
grid_search_xgb = GridSearchCV(xgb, grid_para_xgb, scoring=businessscore, cv=s_kf, n_jobs=-1)
%time grid_search_xgb.fit(X_ros, y_ros)

CPU times: user 6.54 s, sys: 261 ms, total: 6.8 s
Wall time: 25.5 s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_...
                                     n_estimators=100, n_jobs=0,
                                     num_parallel_tree=1, random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_metho

In [283]:
best_xgb = grid_search_xgb.best_estimator_
print('XGB BaseLine Train Set Recall Score:', recall_score(y_ros, best_xgb.predict(X_ros)))
print('XGB BaseLine Validation Set Recall Score:', recall_score(y_test, best_xgb.predict(X_test)))
print('XGB BaseLine Validation Set business Score:', businessgain(y_test, best_xgb.predict(X_test)))
imblearn_xgb = businessgain(y_test, best_xgb.predict(X_test))

XGB BaseLine Train Set Recall Score: 1.0
XGB BaseLine Validation Set Recall Score: 0.9900990099009901
XGB BaseLine Validation Set business Score: 0.5529449490647207


#### comparing SMOTE and imblearn

In [284]:
print('SMOTE xgb business score: ', SMOTE_xgb, '\nimblearn xgb business score: ', imblearn_xgb)

SMOTE xgb business score:  0.47039305782969504 
imblearn xgb business score:  0.5529449490647207


In [285]:
confusion_matrix(y_test, best_xgb.predict(X_test))

array([[519, 462],
       [  1, 100]])